<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://www.cmpe.boun.edu.tr/~ozlem.salehi/" target="_blank"> Özlem Salehi </a> </td>
    </tr> 
 </table>
 
<hr>

# D-Wave Hybrid Solvers

In this notebook, we will learn about the hybrid constrained quadratic solver of D-Wave, which allows modelling problems with constraints, without the need for QUBO formulation.



D-Wave's Leap provides quantum-classical hybrid solvers, which combine classical heuristics with quantum annealing. Those solvers allow tackling larger problem instances compared to QPUs.

Currently, there are three different hybrid solvers: hybrid binary quadratic model (BQM) solver, hybrid constrained quadratic model (CQM) solver, hybrid discrete quadratic model (DQM) solver. We will investigate the first two.

## Hybrid BQM solver

Hybrid BQM solver can be used to solve problems that are expressed in the form of BQM. Using the following code, we can create an instance of hybrid BQM sampler.

In [1]:
from dwave.system import LeapHybridBQMSampler

bqm_solver = LeapHybridBQMSampler()                         

### Solver properties

Let us check some of the properties of the sampler.

In [5]:
bqm_solver.properties.keys()

dict_keys(['minimum_time_limit', 'maximum_time_limit_hrs', 'maximum_number_of_variables', 'maximum_number_of_biases', 'parameters', 'supported_problem_types', 'category', 'version', 'quota_conversion_rate'])

#### Minimum time limit

This parameter gives information about the minimum time solver is required to work on an instance. The first number represents the number of variables and the second one is the time limit in seconds. Linear interpolation should be performed for the number of variables in between.

In [6]:
print("Minimum time limit:", bqm_solver.properties["minimum_time_limit"])

Minimum time limit: [[1, 3.0], [1024, 3.0], [4096, 10.0], [10000, 40.0], [30000, 200.0], [100000, 600.0], [1000000, 600.0]]


#### Maximum time limit 

This is the maximum time allowed in hours.

In [7]:
print("Maximum time limit:", bqm_solver.properties["maximum_time_limit_hrs"])

Maximum time limit: 24.0


#### Maximum number of variables

Maximum number of variables allowed for hybrid BQM solver is 1 million.

In [18]:
print("Maximum number of variables:", bqm_solver.properties["maximum_number_of_variables"])

Maximum number of variables: 1000000


### Running a problem instance

Functions for obtaining the sample, i.e. `sample`, `sample_qubo`, `sample_ising` can still be used with the BQM hybrid solver. Different than the quantum annealer, BQM hybrid solver returns only a single solution. Let's see a simple example.

In [3]:
from dimod import BQM

linear = {'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
quadratic = {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
vartype = 'BINARY'

bqm = BQM(linear, quadratic, vartype)

bqm_solver = LeapHybridBQMSampler()                         
sampleset = bqm_solver.sample(bqm)
print(sampleset)

  x1 x2 x3 x4 energy num_oc.
0  1  0  0  1  -11.0       1
['BINARY', 1 rows, 1 samples, 4 variables]


We can check the time spent on QPU and the total time.

In [4]:
sampleset.info

{'qpu_access_time': 63954,
 'charge_time': 2992762,
 'run_time': 2992762,
 'problem_id': '9d7365dc-b341-4beb-af4b-7c38a88a7b84'}

Here, `qpu_acess_time` is the time spend on QPU, `charge_time` is the time charged to the account and `run_time` is the total run time in microseconds.

### Time limit

We can provide a `time_limit` parameter, which is the maximum run time. It should be larger than the minimum time allowed. If no `time_limit` is provided, it is set to minimum time.

In [5]:
sampleset = bqm_solver.sample(bqm, time_limit = 2)
print(sampleset)

ValueError: time limit for problem size 4 must be at least 3.0

### Task 1

Solve the TSP problem for the following graph with 100 nodes using the hybrid BQM solver. Interpret the results.

In [40]:
import numpy as np
import networkx as nx

np.random.seed(45)
N = 100
G1 = nx.complete_graph(N)
for u, v in G1.edges():
    G1[u][v]["weight"] = np.random.randint(1, 5)

In [ ]:
#
# Your code here
#

[click here for solution](DWave_Hybrid_Solvers_Solutions.ipynb#Task1)

## Hybrid CQM solver

Now we move on to a more interesting solver, that accepts constrained quadratic model.  

Hybrid CQM solver can be initiated as follows.

In [9]:
from dwave.system import LeapHybridCQMSampler

cqm_solver = LeapHybridCQMSampler()                         

To start using the hybrid CQM solver, we should first express our problem as a constrained quadratic model

### Constrained quadratic model

Formally a **constrained quadratic model** (CQM) is defined as 

$$
minimize~~\sum_i a_ix_i + \sum_{i\leq j} b_{ij}x_ix_j + c
$$
$$
subject~to~~ \sum_i a_i^mx_i^m + \sum_{i\leq j} b_{ij}^mx_i^mx_j^m + c^m \circ 0~~m=1,\dots,M
$$
where $x_i$ can be binary, integer or continous variables, $a_i,b_i,c$ are real values, $\circ\in \{=,\leq, \geq\}$ and $M$ is the total number of constraints.

In other words, we have a quadratic function to minimize that is subject to quadratic constraints. This is also known as quadratically constrained quadratic program in the literature.

CQM has several advantages over BQM:
- It is no longer required to obtain a QUBO.
- Constraints are directly specified, no need for the penalty method.
- There is no need to set the penalty values.
- Even quadratic constraints are handled and there is no need for quadratization.
- Integer variables are allowed.

It is also pointed out by D-Wave that hybrid CQM solver performs better than the hybrid BQM solver for constrained problems.

### Formulating CQM using OceanSDK functions

To start with, we will create a simple cqm example. Suppose that we have the following problem.

$$\min.~x_1+3x_3+2x_1x_2-4x_1x_3$$

$$s.t.~x_1x_2 + x_3 = 1$$

$$~~x_2 + x_3 \leq 1$$.
$$x_1+x_3=1$$

### Step 1 - Define an empty CQM and add the cost function you want to minimize


Let's first create the empty CQM.

In [3]:
from dimod import CQM
cqm = CQM()

We will use the function `set_objective` to set the objective value. Objective will be a BQM. Hence, we first create a bqm and set it as our objective.

In [127]:
bqm = BQM("BINARY")
bqm.add_linear("x_1",1)
bqm.add_linear("x_3",3)
bqm.add_quadratic("x_1","x_2", 2)
bqm.add_quadratic("x_1","x_3", -4)

cqm.set_objective(bqm)

Let us check our cqm.

In [128]:
print(cqm)
#print(cqm.variables)
#print(cqm.objective)

Constrained quadratic model: 3 variables, 0 constraints, 5 biases

Objective
  Binary('x_1') + 3*Binary('x_3') - 4*Binary('x_1')*Binary('x_3') + 2*Binary('x_1')*Binary('x_2')

Constraints

Bounds



### Step 2 - Add the Constraints to the CQM

There are multiple ways to add a constraint in CQM. We will use the method `add_constraint` through which you can add linear or quadratic equality or inequality constraints. 

The first way to define constraints is again through first defining a BQM that represents the LHS of the constraint and provide as parameter `(bqm, sense, RHS, label)`. Here, sense is either `<=`, `==` or `>=`. `label` is optional but it is strongly recommende to provide a label for the constraint.

In [129]:
bqm = BQM("BINARY")
bqm.add_quadratic("x_1","x_2",1)
bqm.add_linear("x_3",1)

cqm.add_constraint(bqm, "==", 1, "C1")
print(cqm)

Constrained quadratic model: 3 variables, 1 constraints, 9 biases

Objective
  Binary('x_1') + 3*Binary('x_3') - 4*Binary('x_1')*Binary('x_3') + 2*Binary('x_1')*Binary('x_2')

Constraints
  C1: Binary('x_3') + Binary('x_1')*Binary('x_2') == 1.0

Bounds



Another way to add the constraint is through expressing it using a symbolic math equation where RHS is an integer or a float. This time, we can not use strings to represent variables. We need to create binary variables.

In [130]:
x2, x3 = dimod.Binaries(["x_2", "x_3"])
cqm.add_constraint(x2+x3<=1, "C2")
print(cqm)

Constrained quadratic model: 3 variables, 2 constraints, 11 biases

Objective
  Binary('x_1') + 3*Binary('x_3') - 4*Binary('x_1')*Binary('x_3') + 2*Binary('x_1')*Binary('x_2')

Constraints
  C1: Binary('x_3') + Binary('x_1')*Binary('x_2') == 1.0
  C2: Binary('x_2') + Binary('x_3') <= 1.0

Bounds



In the constrained quadratic model, you can differentiate between **hard** and **soft** constraints. 

Hard constraints are those which should be never violated i.e. a feasbile solution is a solution in which no hard constraint is violated. 

On the other hand, soft constraints may be violated in favor of better solutions overall.

By default, if no parameter is provided, the constraint is a hard constraint. Using the parameter `weight`, you can provide a weight for you constraint which indicates that it is soft. (Unfortunately, no further information is provided about the weight.)

Now, let us suppose that the third constraint is a soft one.

In [131]:
x1 = dimod.Binary("x_1")
cqm.add_constraint(x1+x3==1,"C3", weight = 2)
print(cqm)

Constrained quadratic model: 3 variables, 3 constraints, 13 biases

Objective
  Binary('x_1') + 3*Binary('x_3') - 4*Binary('x_1')*Binary('x_3') + 2*Binary('x_1')*Binary('x_2')

Constraints
  C1: Binary('x_3') + Binary('x_1')*Binary('x_2') == 1.0
  C2: Binary('x_2') + Binary('x_3') <= 1.0
  C3: Binary('x_1') + Binary('x_3') == 1.0

Bounds



In [172]:
print(cqm.constraints["C1"].lhs.is_soft())
print(cqm.constraints["C2"].lhs.is_soft())
print(cqm.constraints["C3"].lhs.is_soft())

False
False
True


### Step 3 - Solve the CQM

We are going to use the `LeapHybridCQMSampler` to solve the CQM and we will use the method `sample_cqm`.

In [136]:
cqm_solver = LeapHybridCQMSampler()                         
sampleset = cqm_solver.sample_cqm(cqm)
print(sampleset)

    x_1 x_2 x_3 energy num_oc. is_sat. is_fea.
95  1.0 0.0 0.0    1.0       1 arra...   False
96  1.0 0.0 0.0    1.0       1 arra...   False
97  1.0 0.0 0.0    1.0       1 arra...   False
98  1.0 0.0 0.0    1.0       1 arra...   False
99  1.0 0.0 0.0    1.0       1 arra...   False
100 1.0 0.0 0.0    1.0       1 arra...   False
101 1.0 0.0 0.0    1.0       1 arra...   False
102 1.0 0.0 0.0    1.0       1 arra...   False
103 1.0 0.0 0.0    1.0       1 arra...   False
104 1.0 0.0 0.0    1.0       1 arra...   False
0   1.0 0.0 1.0    2.0       1 arra...    True
1   1.0 0.0 1.0    2.0       1 arra...    True
3   1.0 0.0 1.0    2.0       1 arra...    True
4   1.0 0.0 1.0    2.0       1 arra...    True
5   1.0 0.0 1.0    2.0       1 arra...    True
6   1.0 0.0 1.0    2.0       1 arra...    True
7   1.0 0.0 1.0    2.0       1 arra...    True
9   1.0 0.0 1.0    2.0       1 arra...    True
10  1.0 0.0 1.0    2.0       1 arra...    True
11  1.0 0.0 1.0    2.0       1 arra...    True
12  1.0 0.0 1

The number of samples returned by the solver can not be determined by the user. The next time you run the problem, you may get a sampleset with different number of samples.


### Step 4 -  Interpret and check the feasibility of the samples in the sampleset and find the optimum sample


In a sampleset obtained from the hybrid CQM solver, note that we have additional fields. Let us start by checking those.

In [182]:
sampleset.info

{'constraint_labels': ['C3', 'C2', 'C1'],
 'qpu_access_time': 31756,
 'charge_time': 5000000,
 'run_time': 5114969,
 'problem_id': 'eabaecaa-fd3d-4880-bf9d-4d4161e5d684'}

In [159]:
for d in sampleset.truncate(5).data():
    print(d)

Sample(sample={'x_1': 1.0, 'x_2': 0.0, 'x_3': 0.0}, energy=1.0, num_occurrences=1, is_satisfied=array([ True,  True, False]), is_feasible=False)
Sample(sample={'x_1': 1.0, 'x_2': 0.0, 'x_3': 0.0}, energy=1.0, num_occurrences=1, is_satisfied=array([ True,  True, False]), is_feasible=False)
Sample(sample={'x_1': 1.0, 'x_2': 0.0, 'x_3': 0.0}, energy=1.0, num_occurrences=1, is_satisfied=array([ True,  True, False]), is_feasible=False)
Sample(sample={'x_1': 1.0, 'x_2': 0.0, 'x_3': 0.0}, energy=1.0, num_occurrences=1, is_satisfied=array([ True,  True, False]), is_feasible=False)
Sample(sample={'x_1': 1.0, 'x_2': 0.0, 'x_3': 0.0}, energy=1.0, num_occurrences=1, is_satisfied=array([ True,  True, False]), is_feasible=False)


We have a field named `is_satisfied` which returns us an array of boolean values indicating whether the constraints are satisfied or not. Note that the order is `C3`, `C2`, `C1`.

We have another field named `is_feasible` which should return `True` in case all hard constraints are satisfied. 

Note that in the lowest energy samples, first constraintis not satisfied, hence the sample is infeasible.

We can filter the feasible samples as follows. In some of those, you can see that the first constraint which is the soft one is not satisfied.

In [183]:
sampleset.filter(lambda d: d.is_feasible).truncate(10)

SampleSet(rec.array([([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True),
           ([1., 0., 1.], 2., 1, [False,  True,  True],  True)],
          dtype=[('sample', '<f8', (3,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('is_satisfied', '?', (3,)), ('is_feasible', '?')]), Variables(['x_1', 'x_2', 'x_3']), {'constraint_labels': ['C3', 'C2', 'C1'], 'qpu_access_time': 31756, 'charge_time': 5000000, 'run_time': 5114969, 'problem_id': 'eabaecaa-fd3d-4880-bf9d-4d4161e5d684'}, 'INTE

### Task 2

Write a funcion named `tsp_cqm` that takes as parameter a networkx graph $G$ and returns the constrained quadratic model for TSP problem. 


In [ ]:
def tsp_cqm(G):
    
    
# Your code here

    

[click here for solution](DWave_Hybrid_Solvers_Solutions.ipynb#Task2)

### Task 3

Using the function you have written in Task 2, obtain the cqm for the given graph and solve it using the hybrid CQM solver.

In [ ]:
import numpy as np
import networkx as nx

np.random.seed(45)
N = 100
G1 = nx.complete_graph(N)
for u, v in G1.edges():
    G1[u][v]["weight"] = np.random.randint(1, 5)

In [ ]:

# Your code here



[click here for solution](DWave_Hybrid_Solvers_Solutions.ipynb#Task3)